In [1]:
from tinyRAG.embedding import VectorStore,read_file_content,MyEmbedding #  我写的RAG，在同个文件夹下
import torch
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from tqdm import tqdm
import json
import re

d:\envs\camel_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resource module not available on Windows


# 所有的路径

In [2]:

BERT_path = r"D:\code\competiton\饿了么智慧点餐\意图识别\bert_binary_classification_model.pth"  # 训练好的意图识别模型路径
BERT_tokenizer_path = r"hfl/chinese-macbert-base"  # 训练好的意图识别模型tokenizer路径
model_dir = r"D:\code\my_work\meet_yourself\Qwen2.5-3B-Instruct"  # Qwen2.5-3B-Instruct模型路径,期待7B的模型
text_path = r'D:\code\competiton\饿了么智慧点餐\RAG推荐\id_text.json' # 测试集语音转文字后的json文件路径格式为[{'text': '天猫精灵来份酸香果仁拌菠菜吧。', 'uuid': '005c01e0-4a95-4230-a2d5-d8dccc93c140'}]
A_path = r'D:\code\competiton\饿了么智慧点餐\RAG推荐\B.txt'# A.txt路径,用于排序
result_path = r'D:\code\competiton\饿了么智慧点餐\RAG推荐\result.txt'# 结果保存路径


In [ ]:
data = []
with open(text_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

data

In [ ]:
uuid_list = []
text_list = []
for item in data:
    uuid_list.append(item['uuid'])
    text_list.append(item['text'])
text_list

In [ ]:


def remove_emojis(text:list[str]) -> list[str]:
    # 定义一个正则表达式模式来匹配所有表情符号
    replace_list = []
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F700-\U0001F77F"  # Alchemical Symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U00002600-\U000026FF"  # Miscellaneous Symbols
        "\U00002700-\U000027BF"  # Dingbats
        "]+", 
        flags=re.UNICODE
    )
    
    for i in text:
        replace_list.append(emoji_pattern.sub(r'', i))
    return replace_list



cleaned_list = remove_emojis(text_list)


In [ ]:
cleaned_list 

In [ ]:
len(cleaned_list)

In [ ]:
with open('cleaned_list.json', 'w', encoding='utf-8') as f:
    json.dump(cleaned_list, f, ensure_ascii=False, indent=4)

# 意图识别模块

In [ ]:
# 意图识别模型加载

from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained(BERT_tokenizer_path)


In [ ]:
model = BertForSequenceClassification.from_pretrained(BERT_tokenizer_path, num_labels=2)
model.load_state_dict(torch.load(BERT_path, map_location=torch.device('cuda')))

model = model.to("cuda")

In [ ]:
with open(r'cleaned_list.json', 'r', encoding='utf-8') as f:
    cleaned_list = json.load(f)

new_texts = cleaned_list

# 数据预处理函数
def tokenize_function_new(examples):
    return tokenizer(examples, padding= 'max_length' , max_length = 64 ,truncation=True, return_tensors='pt')

# 对新文本进行分词和编码
inputs = tokenize_function_new(new_texts)


In [ ]:
ans_One_Zero = []
# 将输入移动到设备（GPU或CPU）
with torch.no_grad():
    for i in cleaned_list:
        inputs = tokenize_function_new(i)
        inputs = {key: value.to('cuda') for key, value in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        ans_One_Zero.append(predictions)
        print(f"Text: '{i}' -> Label: {predictions}")

peridict = []
for i in ans_One_Zero:
    peridict.append(i[0].item())


In [ ]:
len(peridict),len(uuid_list)

In [ ]:
with open(r'意图识别.json', 'w', encoding='utf-8') as f:
    dict_list = []
    for i in  zip(uuid_list,peridict,cleaned_list):
        
        data = {
            'uuid': i[0],
            'text': i[2],
            'perdict': i[1]
        }
        dict_list.append(data)
    json.dump(dict_list, f, ensure_ascii=False,indent=4)


# 意图改写

In [ ]:
import json
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\意图识别.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
data
    
one_text = []
zero = []
one_id = []
for i in data:
    if i['perdict'] == 1: 
        one_id.append(i['uuid'])
        one_text.append(i['text'])
    else:
        zero.append(i)
        
with open(r'zero.json', 'w', encoding='utf-8') as f:
    json.dump(zero, f, ensure_ascii=False, indent=4)

In [ ]:
len(one_id)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
   
    device_map="auto"
    ,torch_dtype=torch.float16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B", device_map="auto", trust_remote_code=True,enable_thinking=False)

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained(model_dir, device_map="auto", trust_remote_code=True)
# `torch_dtype=torch.float16` 可以令模型以 float16 精度加载，否则 modelscope 会将模型加载为 float32，导致显存不足
#,odel = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto",  trust_remote_code=True)
''', torch_dtype=torch.float16'''
g_config = GenerationConfig.from_pretrained(model_dir)
# 显式设置模型的最大长度
g_config.temperature = 2.
g_config.top_p = 0.9
g_config.top_k = 10
model.generation_config = g_config
print(g_config)

In [ ]:
def chat(model, tokenizer, messages, history=[], temperature = 0.1):
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    model_inputs = tokenizer([text], return_tensors="pt").to("cuda")
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        temperature = temperature
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    messages.append({
        "role": "system", "content": response
    })
    
    history = messages

    return response, history

In [ ]:
def dummy_response(example):
        messages = []
        messages.append({"role": "user", 
        "content": 
        f''' 
        角色：你是一位意图识别模型的改写助手，能够根据给出的句子进行意图改写，将句子中有关菜品的部分提取出来。
        任务：根据给定的句子,提取出菜品的名称和要求。其中可能会有方言、错字等等，请识别出菜品名称。如果用户没有给出明确的菜品名称，你要根据用户的要求输出最贴切的菜品。
        生成条件：只能生成1条文本，你要理解样例，并类比样例将有关菜品的部分进行改写。
        禁止：禁止生成重复的内容和问答句子，禁止生成无关内容；输出的文本中不能包含任何多余的内容。
        工作流：参考以下条样例，扮演好你的角色，只需生成一条符合生成条件的文本；
        样例： 
        1.输入： 天猫精灵，来份什锦素菜盖饭 
          输出： 什锦素菜盖饭
        2.输入： 天猫精灵，要碗南瓜小米甜粥
          输出：南瓜小米甜
        3.输入：天猫精灵奶粉热敷的杂属菇烙压桶
          输出：杂属菇烙压桶
        4.输入：来份番茄鸡蛋汤面吧。
          输出：番茄鸡蛋汤面
        5.输入：天猫精灵要碗青菜、肉末粥、热乎的
          输入：青菜、肉末粥
        6.输入：要给老鸭汤分四宝吧。
          输出：老鸭汤分四宝
        以下是你要改写的内容：
        {example}
        注意：你只需要输出菜品名称，只输出一行内容，不输出换行符。不需要其他的内容。请注意，输出的文本中不能包含任何多余的内容。
        ''' })
        response,history = chat(model,tokenizer, messages, history=[])
        print(response+"\n"+example)
        return response

In [ ]:
prompts = one_text
responses = []
for prompt in tqdm(prompts):

    response = dummy_response(prompt)
    responses.append(response)

# Convert responses to a DataFrame


responses为改写结果列表

In [ ]:
with open(r'responses.json', 'w', encoding='utf-8') as f:
    json.dump(responses, f, ensure_ascii=False, indent=4)

In [ ]:
len(responses)

# 调用R1

In [40]:
import json
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\意图识别.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
data
   

[{'uuid': '0021eb2a-594d-41c1-a0e4-4adff49fef29',
  'text': '天猫精灵来粉番茄鸡蛋改饭吧。',
  'perdict': 1},
 {'uuid': '005c067e-2534-4aef-b055-5e5ace8563a5',
  'text': '天猫精灵无限播放小猫的声音。',
  'perdict': 0},
 {'uuid': '00657661-c327-4792-aef0-d558b81fc45d',
  'text': '天猫精灵要个番茄蛋花面疙瘩汤吧。',
  'perdict': 1},
 {'uuid': '008da11f-c860-4910-84a1-f689c1fda8f1',
  'text': '天猫精灵声音调到30唱天猫蓝精灵唱。',
  'perdict': 0},
 {'uuid': '00a575d3-92db-4c1e-bfd6-bd3da612800b',
  'text': '，天猫精灵啊，我在92和70等于多少？',
  'perdict': 0},
 {'uuid': '00bfa8c6-84be-416d-9028-bf9be0b325d6',
  'text': '天猫精灵要个香辣小锅娃娃菜吧。',
  'perdict': 1},
 {'uuid': '010956c5-2cfc-4568-a1ad-aec26b6adc2c',
  'text': '我要唱迪丽热巴的第十幕。',
  'perdict': 0},
 {'uuid': '010afcbf-b02a-4481-867e-6d9c72554755',
  'text': '天猫精灵，请问一加几等于15？',
  'perdict': 0},
 {'uuid': '01439e7d-57d0-4f66-b8af-836cfb0ac5f7',
  'text': '天猫精灵麻烦来份麻酱油麦菜拌演吧。',
  'perdict': 1},
 {'uuid': '0144913e-12ca-4883-9d0b-70352f1cf2f5',
  'text': '天猫精灵要碗红枣红豆粥热乎的。',
  'perdict': 1},
 {'uuid': '014834df-82a4-4913-9bce-

In [41]:
 
one_text = []
zero = []
one_id = []
for i in data:
    if i['perdict'] == 1: 
        one_id.append(i['uuid'])
        one_text.append(i['text'])
    else:
        zero.append(i)
        
with open(r'zero.json', 'w', encoding='utf-8') as f:
    json.dump(zero, f, ensure_ascii=False, indent=4)

In [42]:
len(one_text)

1978

In [ ]:
def dummy_response_R1(example,client):
        messages = []
        messages.append({"role": "user", 
        "content": 
        f''' 
        角色：你是一位意图识别模型的改写助手，能够根据给出的句子进行意图改写，将句子中有关菜品的部分提取出来,并且对给出的句子进行纠错
        任务：根据给定的句子,提取出菜品的名称和要求。其中可能会有方言、错字等等，请识别出菜品名称，对给出的句子进行纠错。如果用户没有给出明确的菜品名称，你要根据用户的要求输出最贴切的菜品。
        生成条件：只能生成1条文本，你要理解样例，并类比样例将有关菜品的部分进行改写，使用\\t对菜品和句子纠错进行分割
        禁止：禁止生成重复的内容和问答句子，禁止生成无关内容；输出的文本中不能包含任何多余的内容。
        工作流：参考以下条样例，扮演好你的角色，只需生成一条符合生成条件的文本；
        样例： 
        1.输入： 天猫精灵，来份什锦素菜盖饭 
          输出： 什锦素菜盖饭\\t天猫精灵，来份什锦素菜盖饭
        2.输入： 天猫精灵，要碗南瓜小米甜粥
          输出：南瓜小米甜粥\\t天猫精灵，要碗南瓜小米甜粥
        3.输入：天猫精灵奶粉热敷的杂属菇烙压桶
          输出：杂属菇烙压桶\\t天猫精灵要份热乎的杂菇老鸭汤
        4.输入：来份番茄鸡蛋汤面吧。
          输出：番茄鸡蛋汤面\\t来份番茄鸡蛋汤面吧
        5.输入：天猫精灵要碗青菜、肉末粥、热乎的
          输入：青菜肉末粥\\t天猫精灵要碗青菜肉末粥热乎的
        6.输入：要给老鸭汤分四宝吧。
          输出：老鸭汤分四宝\\t要给老鸭汤分四宝吧
        7.输入：天猫精灵青菜豆腐汤来一份热乎的就行。
          输出：青菜豆腐汤\\t天猫精灵青菜豆腐汤来一份热乎的就行
        以下是你要改写的内容：
        {example}
        注意：你只需要输出菜品名称和对给出的句子进行纠错，使用\\t对菜品和句子纠错进行分割，只输出一行内容，不输出换行符。不需要其他的内容。请注意，输出的文本中不能包含任何多余的内容。
        ''' })
        response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
        response =  response.choices[0].message.content
        print(response+"\n"+example)
        return response

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='sk-30d9e5ff6f22478687ae7d87f6615edc', base_url="https://api.deepseek.com")




In [38]:
for prompt in tqdm(prompts[1419:]):

    response = dummy_response_R1 (prompt,client)
    responses.append(response)

# Convert responses to a DataFrame

 73%|███████▎  | 409/559 [34:38<11:55,  4.77s/it]

番茄炒蛋盖饭\t天猫精灵要份番茄炒蛋盖饭吧
天猫精灵要份番茄炒蛋盖饭吧。


 73%|███████▎  | 410/559 [34:42<11:25,  4.60s/it]

天麻乌鸡汤\t天猫精灵，要份天麻乌鸡汤吧
天猫精灵，要粉天马两只无鸡杨肾烫吧。


 74%|███████▎  | 411/559 [34:47<11:43,  4.75s/it]

...

山药肉粥\t天猫精灵来碗山药肉粥热乎的
天猫精灵来我山药肉粥热敷达。


100%|█████████▉| 558/559 [46:56<00:04,  4.81s/it]

韭黄炒蛋\t天猫精灵来份韭黄炒蛋
天猫精灵来份韭黄炒蛋。


100%|██████████| 559/559 [47:00<00:00,  5.05s/it]

香菇青菜肉末粥\t天猫精灵要碗香菇青菜肉末粥吧
天猫精灵要碗香菇青菜肉末粥吧。


In [43]:
responses

['番茄鸡蛋盖饭\\t天猫精灵来份番茄鸡蛋盖饭吧',
 '番茄蛋花面疙瘩汤\\t天猫精灵要个番茄蛋花面疙瘩汤吧',
 '香辣小锅娃娃菜\\t天猫精灵要个香辣小锅娃娃菜吧',
 '麻酱油麦菜拌面\\t天猫精灵麻烦来份麻酱油麦菜拌面吧。',
 '红枣红豆粥\\t天猫精灵要碗红枣红豆粥热乎的',
 '香菇瘦肉热粥\\t天猫精灵要碗香菇瘦肉热粥吧',
 '皮蛋瘦肉粥\\t天猫精灵要碗香浓皮蛋瘦肉粥吧',
 '山药炒木耳\\t天猫精灵来份山药炒木耳清淡的',
 '山药羊排养生汤\\t天猫精灵要份山药羊排养生汤吧',
 '鲜炖鱼头豆腐汤\\t天猫精灵来份鲜炖鱼头豆腐汤吧',
 '酸辣白菜\\t天猫精灵帮我来个酸辣白菜',
 '南瓜小米粥\\t天猫精灵要碗南瓜小米粥',
 '皮蛋瘦肉粥\\t天猫精灵要碗热乎的皮蛋瘦肉粥',
 '皮蛋瘦肉粥\\t天猫精灵要一份暖心暖胃的皮蛋瘦肉粥吧',
 '小米粥\\t天猫精灵来份暖心养胃的小米粥吧',
 '鲜虾仁热粥\\t天猫精灵来碗鲜虾仁的热粥吧',
 '豆腐皮裹香菜\\t天猫精灵要个豆腐皮裹香菜吧',
 '小白菜炒千张丝\\t天猫精灵要个小白菜炒千张丝吧',
 '来往县疙瘩汤\\t天猫精灵来碗现做的疙瘩汤',
 '四季豆炒橄榄菜\\t天猫精灵来份四季豆炒橄榄菜吧',
 '香菇肉末粥\\t天猫精灵来碗香菇肉末粥吧',
 '小米粥\\t天猫精灵来碗热乎养胃的小米粥吧',
 '鲜格力面疙瘩汤\\t天猫精灵来碗鲜葛面疙瘩汤吧',
 '包菜粉丝炒蛋\\t要个包菜粉丝炒蛋吧',
 '杂后的皮蛋瘦肉粥\\t天猫精灵来碗热乎的皮蛋瘦肉粥',
 '土豆片烩嫩牛肉\\t天猫精灵要份土豆片烩嫩牛肉',
 '红枣南瓜小米粥\\t天猫精灵来碗红枣南瓜小米粥吧',
 '西红柿鸡蛋浇头饭\\t来份西红柿鸡蛋浇头饭',
 '酸苗炒肉丝\\t要个酸苗炒肉丝吧。',
 '萝卜炖排骨汤\\t来份萝卜炖排骨汤',
 '小米粥\\t天猫精灵来碗暖胃的小米粥吧',
 '空心菜\\t天猫精灵来份新鲜现炒的空心菜吧',
 '排骨小青菜泡饭\\t来一份排骨小青菜泡饭，热腾腾，味道老好。今朝就点半个。',
 '养胃玉米粥\\t天猫精灵来碗热乎的养胃玉米粥吧。',
 '粥热乎的\\t天猫精灵要碗热乎的粥',
 '蒜香油麦菜\\t要个蒜香油麦菜',
 '红枣红豆软心粥\\t天猫精灵来碗红枣红豆软心

In [45]:
for i in range(len(responses)):
    if '\\t' not in responses[i]:
        print(i,responses[i])

892 艾


In [56]:
one_text_ds = []
if len(one_text) == len(responses):
    for i in range(len(responses)):
        if '\\t' in responses[i]:
            one_text_ds.append(responses[i].split('\\t')[1])
        else:
            one_text_ds.append(one_text[i])


In [57]:
one_text_ds

['天猫精灵来份番茄鸡蛋盖饭吧',
 '天猫精灵要个番茄蛋花面疙瘩汤吧',
 '天猫精灵要个香辣小锅娃娃菜吧',
 '天猫精灵麻烦来份麻酱油麦菜拌面吧。',
 '天猫精灵要碗红枣红豆粥热乎的',
 '天猫精灵要碗香菇瘦肉热粥吧',
 '天猫精灵要碗香浓皮蛋瘦肉粥吧',
 '天猫精灵来份山药炒木耳清淡的',
 '天猫精灵要份山药羊排养生汤吧',
 '天猫精灵来份鲜炖鱼头豆腐汤吧',
 '天猫精灵帮我来个酸辣白菜',
 '天猫精灵要碗南瓜小米粥',
 '天猫精灵要碗热乎的皮蛋瘦肉粥',
 '天猫精灵要一份暖心暖胃的皮蛋瘦肉粥吧',
 '天猫精灵来份暖心养胃的小米粥吧',
 '天猫精灵来碗鲜虾仁的热粥吧',
 '天猫精灵要个豆腐皮裹香菜吧',
 '天猫精灵要个小白菜炒千张丝吧',
 '天猫精灵来碗现做的疙瘩汤',
 '天猫精灵来份四季豆炒橄榄菜吧',
 '天猫精灵来碗香菇肉末粥吧',
 '天猫精灵来碗热乎养胃的小米粥吧',
 '天猫精灵来碗鲜葛面疙瘩汤吧',
 '要个包菜粉丝炒蛋吧',
 '天猫精灵来碗热乎的皮蛋瘦肉粥',
 '天猫精灵要份土豆片烩嫩牛肉',
 '天猫精灵来碗红枣南瓜小米粥吧',
 '来份西红柿鸡蛋浇头饭',
 '要个酸苗炒肉丝吧。',
 '来份萝卜炖排骨汤',
 '天猫精灵来碗暖胃的小米粥吧',
 '天猫精灵来份新鲜现炒的空心菜吧',
 '来一份排骨小青菜泡饭，热腾腾，味道老好。今朝就点半个。',
 '天猫精灵来碗热乎的养胃玉米粥吧。',
 '天猫精灵要碗热乎的粥',
 '要个蒜香油麦菜',
 '天猫精灵来碗红枣红豆软心粥',
 '天猫精灵来份木耳炒泡菜吧',
 '来碗热乎的土鸡汤吧',
 '天猫精灵来碗红豆薏米养胃粥吧',
 '天猫精灵要份热乎的皮蛋瘦肉粥',
 '天猫精灵来份米粉',
 '哎，帮我来一份金灿灿小米养生粥，养养胃蛮好。',
 '天猫精灵要个酸黄瓜丝小炒吧。',
 '天猫精灵要份家常紫菜蛋花汤',
 '来碗暖暖的小米粥吧',
 '有煮得软的吗？',
 '天猫精灵要个什锦鲜蔬小炒吧。',
 '天猫精灵来碗热乎的招牌皮蛋瘦肉粥',
 '天猫精灵来份蘑菇小炒肉吧',
 '天猫精灵来碗热乎的皮蛋瘦肉粥吧',
 '天猫精灵要碗热乎的紫菜蛋花汤',
 '天猫精灵来碗玉米养生汤',
 '天猫精灵来碗热乎营养八宝粥吧',
 '帮我来一份小米南瓜养胃粥，味

In [55]:
with open(r'one_text_ds.json', 'w', encoding='utf-8') as f:
    json.dump(one_text_ds, f, ensure_ascii=False, indent=4)

# RAG模块

In [16]:
import json
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\one_text_ds.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [17]:
len(data)

1978

In [18]:
# 去除字符串列表的所有表情、标点，只保留汉字
import re

def remove_punctuation_and_emoji(strings):
    result = []
    for string in strings:
        # 使用正则表达式匹配汉字
        cleaned_string = re.sub(r'[^\u4e00-\u9fa5]', '', string)
        result.append(cleaned_string)
    return result

data = remove_punctuation_and_emoji(data)
data

['番茄鸡蛋盖饭',
 '番茄蛋花面疙瘩汤',
 '香辣小锅娃娃菜',
 '麻酱油麦菜拌面',
 '红枣红豆粥',
 '香菇瘦肉热粥',
 '皮蛋瘦肉粥',
 '山药炒木耳',
 '山药羊排养生汤',
 '鲜炖鱼头豆腐汤',
 '酸辣白菜',
 '南瓜小米粥',
 '皮蛋瘦肉粥',
 '皮蛋瘦肉粥',
 '小米粥',
 '鲜虾仁热粥',
 '豆腐皮裹香菜',
 '小白菜炒千张丝',
 '来往县疙瘩汤',
 '四季豆炒橄榄菜',
 '香菇肉末粥',
 '小米粥',
 '鲜格力面疙瘩汤',
 '包菜粉丝炒蛋',
 '杂后的皮蛋瘦肉粥',
 '土豆片烩嫩牛肉',
 '红枣南瓜小米粥',
 '西红柿鸡蛋浇头饭',
 '酸苗炒肉丝',
 '萝卜炖排骨汤',
 '小米粥',
 '空心菜',
 '排骨小青菜泡饭',
 '养胃玉米粥',
 '粥热乎的',
 '蒜香油麦菜',
 '红枣红豆软心粥',
 '木耳炒泡菜',
 '土鸡汤',
 '红豆薏米养胃粥',
 '皮蛋瘦肉粥',
 '米粉',
 '小米养生粥',
 '酸黄瓜丝小草',
 '家常紫菜蛋花汤',
 '小米粥',
 '煮软的',
 '石令先蔬小草',
 '招牌皮蛋瘦肉粥',
 '蘑菇小炒肉',
 '皮蛋瘦肉粥',
 '紫菜蛋花汤',
 '玉米养生汤',
 '八宝粥',
 '小米南瓜养胃粥',
 '番茄蛋花汤',
 '海米冬瓜汤',
 '番茄蛋花汤',
 '蘑菇汤面',
 '话鸡蛋小炒',
 '清汤',
 '擎载粥',
 '皮蛋瘦肉粥',
 '清炒小白菜',
 '家常白菜炖豆腐',
 '白菜炖冻豆腐',
 '香菇鸡丝粥',
 '家常紫菜蛋花汤',
 '虫草花瘦肉汤',
 '萝卜炖牛肉汤',
 '三鲜豆腐汤',
 '鲜鱼头炖豆腐汤',
 '小米南瓜热粥',
 '皮蛋瘦肉粥',
 '家常土豆丝',
 '皮蛋瘦肉粥',
 '蚕豆木耳炒蛋',
 '健康菜品',
 '红糖姜茶',
 '山药鸡茸养胃粥',
 '皮蛋瘦肉粥',
 '红豆八宝粥',
 '番茄炒蛋',
 '红豆花生枣粥',
 '酸菜炒粉条',
 '番茄炒蛋',
 '白米饭',
 '南瓜小米粥',
 '黑木耳炒肉',
 '软烂易嚼食物',
 '容易下咽的粥',
 '八宝粥',
 '青椒炒肉盖饭',
 '皮蛋鱼头豆腐汤',
 '皮蛋瘦肉粥',
 '番茄炒蛋配米饭',
 '南瓜小米养

In [6]:
model = 'google-bert/bert-base-chinese'
embedding_model = MyEmbedding(path=model)
vs = VectorStore()
vs.load_vector('数据库')
# 查询示例
query = '鱼豆腐汤'
result = vs.query(query = query,EmbeddingModel=embedding_model,k=10)
result

No sentence-transformers model found with name google-bert/bert-base-chinese. Creating a new one with mean pooling.


['大鱼头豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '黑鱼豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '鳕鱼豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '鱼丸海带豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '鱼头豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '水鱼豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '青鱼头豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '黄古鱼豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '昂刺鱼豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹',
 '鱼排豆腐汤[SEP]汤羹[SEP]菜品[SEP]汤羹']

In [19]:
# 搜索
searchResults = []
for i in data:
    result = vs.query(query = i,EmbeddingModel=embedding_model,k=5)
    print(result,i)
    searchResults.append(result)

['番茄鸡蛋肉末盖饭[SEP]盖浇饭[SEP]主食[SEP]饭[SEP]盖浇饭', '番茄炒鸡蛋盖饭[SEP]盖浇饭[SEP]主食[SEP]饭[SEP]盖浇饭', '蕃茄鸡蛋盖饭[SEP]盖浇饭[SEP]主食[SEP]饭[SEP]盖浇饭', '番茄鸡蛋盖饭[SEP]盖浇饭[SEP]主食[SEP]饭[SEP]盖浇饭', '西红柿炒鸡蛋盖饭[SEP]盖浇饭[SEP]其他[SEP]饭[SEP]盖浇饭'] 番茄鸡蛋盖饭
['番茄豆腐蛋花疙瘩汤[SEP]汤羹[SEP]菜品[SEP]汤羹', '番茄蛋花疙瘩汤[SEP]汤羹[SEP]菜品[SEP]汤羹', '番茄酱鸡蛋疙瘩汤[SEP]汤羹[SEP]菜品[SEP]汤羹', '高汤番茄西红柿蛋花汤[SEP]汤羹[SEP]菜品[SEP]汤羹', '西红柿蛋花疙瘩汤[SEP]汤羹[SEP]菜品[SEP]汤羹'] 番茄蛋花面疙瘩汤
['蒜蓉金菇娃娃菜[SEP]热菜[SEP]菜品[SEP]热菜', '樱花虾拌娃娃菜[SEP]凉菜[SEP]菜品[SEP]凉菜', '蒜蓉粉丝蒸娃娃菜[SEP]热菜[SEP]菜品[SEP]热菜', '蛰头拌娃娃菜[SEP]凉菜[SEP]菜品[SEP]凉菜', '蒜蓉炒娃娃菜[SEP]热菜[SEP]菜品[SEP]热菜'] 香辣小锅娃娃菜
['麻酱拌油麦菜[SEP]凉菜[SEP]菜品[SEP]凉菜', '姥姥葱油拌面[SEP]拌面[SEP]主食[SEP]面[SEP]拌面', '热拌青椒干丝面[SEP]拌面[SEP]主食[SEP]面[SEP]拌面', '外婆葱油拌面[SEP]拌面[SEP]主食[SEP]面[SEP]拌面', '葱油双菇拌面[SEP]拌面[SEP]主食[SEP]面[SEP]拌面'] 麻酱油麦菜拌面
['红糖红豆红枣粥[SEP]粥[SEP]主食[SEP]粥', '红豆红枣红糖粥[SEP]粥[SEP]主食[SEP]粥', '红豆红枣芋圆粥[SEP]粥[SEP]主食[SEP]粥', '红豆薏米红枣粥[SEP]粥[SEP]主食[SEP]粥', '冰糖红枣红豆粥[SEP]粥[SEP]主食[SEP]粥'] 红枣红豆粥
['砂锅香菇瘦肉粥[SEP]粥[SEP]主食[SEP]粥', '香菇干贝瘦肉粥[SEP]粥[SEP]主食[SEP]粥', '香菇小肠瘦肉粥[SEP]粥[SEP]主食[SEP]粥', '香菇青菜鸡肉粥[S

KeyboardInterrupt: 

searchResults为搜索结果，更改K值可以更改搜索结果的数量

In [20]:
with open(r'searchResults.json', 'w', encoding='utf-8') as f:
    json.dump(searchResults, f, ensure_ascii=False, indent=4)

In [21]:
results = []
for i in searchResults:
    results.append(i[0].split('[SEP]')[0])
 


In [22]:
len(results)

36

In [8]:
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\意图识别.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [9]:
data

[{'uuid': '0021eb2a-594d-41c1-a0e4-4adff49fef29',
  'text': '天猫精灵来粉番茄鸡蛋改饭吧。',
  'perdict': 1},
 {'uuid': '005c067e-2534-4aef-b055-5e5ace8563a5',
  'text': '天猫精灵无限播放小猫的声音。',
  'perdict': 0},
 {'uuid': '00657661-c327-4792-aef0-d558b81fc45d',
  'text': '天猫精灵要个番茄蛋花面疙瘩汤吧。',
  'perdict': 1},
 {'uuid': '008da11f-c860-4910-84a1-f689c1fda8f1',
  'text': '天猫精灵声音调到30唱天猫蓝精灵唱。',
  'perdict': 0},
 {'uuid': '00a575d3-92db-4c1e-bfd6-bd3da612800b',
  'text': '，天猫精灵啊，我在92和70等于多少？',
  'perdict': 0},
 {'uuid': '00bfa8c6-84be-416d-9028-bf9be0b325d6',
  'text': '天猫精灵要个香辣小锅娃娃菜吧。',
  'perdict': 1},
 {'uuid': '010956c5-2cfc-4568-a1ad-aec26b6adc2c',
  'text': '我要唱迪丽热巴的第十幕。',
  'perdict': 0},
 {'uuid': '010afcbf-b02a-4481-867e-6d9c72554755',
  'text': '天猫精灵，请问一加几等于15？',
  'perdict': 0},
 {'uuid': '01439e7d-57d0-4f66-b8af-836cfb0ac5f7',
  'text': '天猫精灵麻烦来份麻酱油麦菜拌演吧。',
  'perdict': 1},
 {'uuid': '0144913e-12ca-4883-9d0b-70352f1cf2f5',
  'text': '天猫精灵要碗红枣红豆粥热乎的。',
  'perdict': 1},
 {'uuid': '014834df-82a4-4913-9bce-

In [10]:
one_text = []
zero = []
one_id = []
one_label = []
for i in data:
    if i['perdict'] == 1: 
        one_id.append(i['uuid'])
        one_text.append(i['text'])
        one_label.append(i['perdict'])
    else:
        zero.append(i)

In [18]:
one_text

['天猫精灵来粉番茄鸡蛋改饭吧。',
 '天猫精灵要个番茄蛋花面疙瘩汤吧。',
 '天猫精灵要个香辣小锅娃娃菜吧。',
 '天猫精灵麻烦来份麻酱油麦菜拌演吧。',
 '天猫精灵要碗红枣红豆粥热乎的。',
 '天猫精灵要碗香菇瘦肉热粥吧。',
 '天猫精灵要碗香浓皮蛋瘦肉粥吧。',
 '天猫精灵来份山药炒木耳清淡的。',
 '天猫精灵要分山药羊排养生汤吧。',
 '天猫精灵来份鲜炖鱼头豆腐汤吧。',
 '天猫精灵帮我来个酸酸脆脆的白菜。',
 '天猫精灵要换杨小米男瓜作。',
 '天猫精灵要吻热乎的皮蛋瘦入粥。',
 '天猫精灵要一份暖心暖胃的皮蛋瘦肉粥吧。',
 '天猫精灵来份暖心养胃的小米粥吧。',
 '天猫精灵来玩鲜熟虾仁的热粥吧。',
 '天猫精灵要个豆腐皮裹香菜吧。',
 '天猫精灵要个小白菜炒千张丝吧。',
 '天猫精灵来往县给里面疙搭他。',
 '天猫精灵来份四季豆炒橄榄菜吧。',
 '天猫精灵来碗香菇肉没粥吧。',
 '天猫精灵来碗热乎养胃的小米粥吧。',
 '天猫精灵来碗鲜格力面疙瘩汤吧。',
 '要个包菜粉丝炒蛋吧。',
 '天猫精的来完咋后的皮单手肉咒。',
 '天猫精灵要份土豆片烩嫩牛肉。',
 '天猫精灵来碗红枣南瓜小米粥吧。',
 '来份西红柿鸡蛋浇头饭。',
 '要个酸苗炒肉食吧。',
 '来份萝卜炖排骨汤。',
 '天猫精灵来碗暖胃的小米粥吧。',
 '天猫精灵来份新鲜现炒的空心菜吧。',
 '拉要一份排骨小青菜泡饭，热腾腾，味道老好。今朝就点半个。',
 '天猫精灵来玩乐湖的养威玉米粥吧。',
 '天猫精灵要周乐乎的。',
 '要个蒜香油木菜。',
 '天猫精灵奶碗红枣红豆软心粥。',
 '天猫精灵来份木耳草泡泡菜吧。',
 '来碗热乎的土鸡汤吧。',
 '天猫精灵来碗红豆薏米养胃粥吧。',
 '天猫精灵要份热乎的皮蛋瘦肉走吧。',
 '天猫精灵life粉。',
 '哎，帮我来一份金灿灿小米养生照。好，养养胃蛮好。',
 '天猫精灵要个酸黄瓜丝小草吧。',
 '天猫精灵要份家常紫菜蛋花汤。',
 '来碗暖暖的小米粥吧。',
 '有煮的软的吗？',
 '天猫精灵要个石令先蔬小草吧。',
 '天猫精灵来碗热乎的招牌皮蛋瘦肉粥。',
 '天猫精灵来份蘑菇小炒肉吧。',
 '天猫精灵来碗热乎的皮蛋瘦肉粥吧。',
 '天猫精灵要碗热乎的紫菜蛋花

In [ ]:
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\responses_deepseek_V3.json', 'r', encoding='utf-8') as f:
    responses = json.load(f)

In [3]:
one_text_ds = []
if len(one_text) == len(responses):
    for i in range(len(responses)):
        if '\\t' in responses[i]:
            one_text_ds.append(responses[i].split('\\t')[1])
        else:
            one_text_ds.append(one_text[i])

NameError: name 'one_text' is not defined

In [27]:
one_text = one_text_ds

In [ ]:
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\zero.json', 'r', encoding='utf-8') as f1:
        zero = json.load(f1)
def dummy_response_R1(example,client):
        messages = []
        messages.append({"role": "user", 
        "content": 
        f''' 
        角色：你是一位语音识别后的文字的改写助手，能够根据给出的句子进行纠错；
        任务：根据给定的句子，其中可能会有方言、错字等等，对给出的句子进行纠错。
        生成条件：只能生成1条文本，你要理解样例，对句子纠错。
        禁止：禁止生成重复的内容和问答句子，禁止生成无关内容；输出的文本中不能包含任何多余的内容。
        工作流：参考以下条样例，扮演好你的角色，只需生成一条符合生成条件的文本；
        样例： 
        1.输入： 天猫精灵声音调到30唱天猫蓝精灵唱。
          输出： 天猫精灵声音调到30唱天猫精灵唱。
        2.输入： 天猫精灵，要碗南瓜小米甜粥
          输出：南瓜小米甜粥\\t天猫精灵，要碗南瓜小米甜粥
        3.输入：天猫精灵奶粉热敷的杂属菇烙压桶
          输出：杂属菇烙压桶\\t天猫精灵要份热乎的杂菇老鸭汤
        4.输入：来份番茄鸡蛋汤面吧。
          输出：番茄鸡蛋汤面\\t来份番茄鸡蛋汤面吧
        5.输入：天猫精灵要碗青菜、肉末粥、热乎的
          输入：青菜肉末粥\\t天猫精灵要碗青菜肉末粥热乎的
        6.输入：要给老鸭汤分四宝吧。
          输出：老鸭汤分四宝\\t要给老鸭汤分四宝吧
        7.输入：天猫精灵青菜豆腐汤来一份热乎的就行。
          输出：青菜豆腐汤\\t天猫精灵青菜豆腐汤来一份热乎的就行
        以下是你要改写的内容：
        {example}
        注意：你只需要输出菜品名称和对给出的句子进行纠错，使用\\t对菜品和句子纠错进行分割，只输出一行内容，不输出换行符。不需要其他的内容。请注意，输出的文本中不能包含任何多余的内容。
        ''' })
        response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
        response =  response.choices[0].message.content
        print(response+"\n"+example)
        return response

In [12]:
zero

[{'uuid': '005c067e-2534-4aef-b055-5e5ace8563a5',
  'text': '天猫精灵无限播放小猫的声音。',
  'perdict': 0},
 {'uuid': '008da11f-c860-4910-84a1-f689c1fda8f1',
  'text': '天猫精灵声音调到30唱天猫蓝精灵唱。',
  'perdict': 0},
 {'uuid': '00a575d3-92db-4c1e-bfd6-bd3da612800b',
  'text': '，天猫精灵啊，我在92和70等于多少？',
  'perdict': 0},
 {'uuid': '010956c5-2cfc-4568-a1ad-aec26b6adc2c',
  'text': '我要唱迪丽热巴的第十幕。',
  'perdict': 0},
 {'uuid': '010afcbf-b02a-4481-867e-6d9c72554755',
  'text': '天猫精灵，请问一加几等于15？',
  'perdict': 0},
 {'uuid': '014834df-82a4-4913-9bce-c0c42807784d',
  'text': '我想亲你一亲一下你。',
  'perdict': 0},
 {'uuid': '0155d522-b5f8-4cb5-9d28-f4cb4723a3d7',
  'text': '天猫精灵2404乘7.24。',
  'perdict': 0},
 {'uuid': '019a240b-2d23-4ec1-8a96-826790949e53',
  'text': '天猫精灵人在晃动的时候，大脑会晃动吗？',
  'perdict': 0},
 {'uuid': '01a8ba96-6b92-4165-8657-a9e325f1e90a',
  'text': '生中蛋白玉米怎么搭配营养天呢？',
  'perdict': 0},
 {'uuid': '01e92047-d379-4c91-a39f-c4810561c443',
  'text': '天猫精灵上海华山医院徐雷。',
  'perdict': 0},
 {'uuid': '01eba133-e066-4c37-97c5-d55ebb

In [28]:

with open('ans_1.txt','w',encoding='utf-8') as f:
    for i in zip(one_id,one_text,results):
        f.write(i[0]+'\t'+i[1]+'\t'+'1'+'\t'+i[2]+'\n')
    with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\zero.json', 'r', encoding='utf-8') as f1:
        zero = json.load(f1)
    for i in zero:
        f.write(i['uuid']+'\t'+i['text']+'\t'+'0'+'\t'+'\n')

In [31]:
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\ans_1.txt', 'r', encoding='utf-8') as f:
    ans = []
    for line in f:
        ans.append(line.strip())
ans1 = dict()
for i in ans:
    ans1[i.split('\t')[0]] = i
ans1
len(ans)
with open(r'D:\code\competiton\饿了么智慧点餐\RAG推荐\B.txt', 'r', encoding='utf-8') as f:
    A = []
    for line in f:
        A.append(line.strip())
A = A[1:]

with open(result_path, 'w', encoding='utf-8') as f:
    for line in A:
        f.write(ans1[line] + '\n')